In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory that gets preserved as output when you create a version using "Save & Run All"

# **Hateful Meme Detection**

**CHINTHA YETHI RAJ**
**210108013**

In recent years, the proliferation of hateful content on social media platforms has emerged as a significant concern, necessitating robust detection mechanisms to safeguard online communities. This project aims to develop an advanced system for the detection of hateful memes, which are a particularly insidious form of harmful content combining images and text to convey hate speech or discrimination. The proposed solution leverages state-of-the-art machine learning techniques, including convolutional neural networks (CNNs) for image analysis and transformer-based models for natural language processing (NLP). By integrating these methodologies, the system can accurately interpret and assess the multimodal content of memes.

The core of the project involves creating a comprehensive dataset of labeled hateful and non-hateful memes, ensuring diversity in both visual and textual components to enhance the model's generalizability. We employ a multi-stage training process where the CNNs extract visual features, and the transformer models analyze textual data, followed by a fusion mechanism to combine these features for final classification. Additionally, we implement techniques to address challenges such as context variability, sarcasm, and subtlety in hate speech, which are prevalent in memes.

Performance evaluation of the system is conducted using standard metrics such as accuracy, precision, recall, and F1-score, along with a comparative analysis against existing state-of-the-art approaches. Preliminary results demonstrate the effectiveness of our model in accurately detecting hateful memes, highlighting its potential for deployment in real-world social media monitoring tools.

The project's contributions include not only a robust detection model but also insights into the challenges and nuances of multimodal hate speech detection. By enhancing the ability to automatically identify and mitigate the spread of hateful content, this project aims to contribute significantly to the creation of safer online environments.

## Table of Content

[Problem](#problem)   
[Data Understanding](#data_understanding)   
[Data Preparation](#data_preparation)   
[Modeling](#modeling)   
[Evaluation](#evaluation)   
[References](#references)   

## Problem <a class="anchor" id="problem"></a>

The rise of memes on social media has turned them into influential communication tools. Unfortunately, this medium is also being used to spread hate speech and harmful content. Detecting hateful memes is challenging due to their multimodal nature, where harmful intent can be subtly embedded in both images and text. Traditional detection systems focusing only on text or images are insufficient for understanding these nuances.

Key Questions Addressed in the Project

1. How can we effectively detect hate speech in memes?

    Memes require an integrated approach to understand the interplay between visual and textual elements.
    
2. What machine learning models and techniques are most effective for this task?

    Exploring and combining CNNs for image processing and transformers for text analysis to create an accurate classification system.
    
3. How can we create a comprehensive dataset for training and evaluation?

    Collecting and annotating a diverse set of memes reflecting real-world variability for robust model training.
    
4. What challenges arise in detecting subtle and context-dependent hate speech in memes?

    Addressing issues like sarcasm, irony, and cultural references that obscure harmful intent.
    
5. How can we evaluate the performance of the detection system?

    Using metrics like accuracy, precision, recall, and F1-score, and comparing results with existing methods.
    
6. What are the ethical considerations and potential biases in detecting hateful content?

    Ensuring fairness, avoiding censorship of benign content, and addressing biases in training data.
    
This project aims to develop an effective hateful meme detection system to enhance online safety and inclusivity.

## Data Understanding<a class="anchor" id="data_understanding"></a>

For this project, we will utilize the Facebook Hateful Memes Detection Challenge dataset. This dataset is specifically designed for multimodal hate speech detection and includes a diverse collection of memes, combining images and associated text. Each meme is labeled as either hateful or non-hateful. The dataset provides a balanced mix of various cultural references and contexts, ensuring comprehensive coverage for training and evaluation. By leveraging this dataset, we aim to build and validate a robust system capable of accurately detecting hateful content in memes.

In [ ]:
%matplotlib inline

import json
import logging
from pathlib import Path
import random
import tarfile
import tempfile
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torchvision

In [ ]:
data_dir = "/kaggle/input/data-intensive/data/"
train_path = data_dir + "train.jsonl"
dev_path = data_dir  + "dev.jsonl"
test_path = data_dir + "test.jsonl"

In [ ]:
train_samples_frame = pd.read_json(train_path, lines=True)
val_samples_frame = pd.read_json(dev_path, lines=True)
train_samples_frame.head()

In [ ]:
train_samples_frame.label.value_counts()

In [ ]:
train_samples_frame['length'] = train_samples_frame['text'].apply(len)
# Define the number of bins
num_bins = 3

# Create bins automatically
train_samples_frame['length_bins'] = pd.cut(train_samples_frame['length'], bins=num_bins)


In [ ]:
import seaborn as sns

# Using Seaborn for better aesthetics
plt.figure(figsize=(10, 6))
sns.countplot(x='length_bins', data=train_samples_frame, palette='viridis')

plt.title('Number of Sentences in Length Ranges')
plt.xlabel('Sentence Length Range')
plt.ylabel('Number of Sentences')
plt.xticks(rotation=45)
plt.show()


In [ ]:
max_length = train_samples_frame['length'].max()
print("Maximum Length:", max_length)
median_length = train_samples_frame['length'].median()
print("Median Length:", median_length)


In [ ]:
train_samples_frame

## Data Preparation<a class="anchor" id="data_preparation"></a>

Explain what kind of data transformations, feature selection and/or engineering you will perform.

In [ ]:
train_samples_frame = train_samples_frame[train_samples_frame['length'] < 64]

In [ ]:
train_samples_frame = train_samples_frame.sample(2000)

In [ ]:
train_texts = list(train_samples_frame['text'])
val_texts = list(val_samples_frame['text'])
train_labels = list(train_samples_frame['label'])
val_labels = list(val_samples_frame['label'])

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
train_images = []
val_images = []

for i in list(val_samples_frame['img']):
    path = data_dir + i
    img = load_img(path, target_size=(128, 128))
    img_array = img_to_array(img)
    img_array /= 255.0  # Normalize pixel values
    val_images.append(img_array)

for i in list(train_samples_frame['img']):
    path = data_dir + i
    img = load_img(path, target_size=(128, 128))
    img_array = img_to_array(img)
    img_array /= 255.0  # Normalize pixel values
    train_images.append(img_array)

train_images = np.array(train_images)
val_images = np.array(val_images)

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate, GlobalAveragePooling2D,Flatten
from transformers import TFBertModel, BertTokenizer
import numpy as np

# Load pre-trained BERT model and tokenizer
bert_model = TFBertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def get_bert_embeddings(texts):
    inputs = tokenizer(texts, return_tensors="tf", padding=True, truncation=True, max_length=128)
    outputs = bert_model(inputs)
    outputs = outputs.last_hidden_state[:, 0, :]  # Use the [CLS] token embedding
    return outputs



In [ ]:
train_text_embeddings = get_bert_embeddings(train_texts)
val_text_embeddings = get_bert_embeddings(val_texts)
train_text_embeddings = np.array(train_text_embeddings)
val_text_embeddings = np.array(val_text_embeddings)

In [ ]:
train_text_embeddings.shape

In [ ]:
train_labels = np.array(train_labels)
val_labels = np.array(val_labels)

## Modeling<a class="anchor" id="modeling"></a>

In [ ]:
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import Input, Dense, Concatenate, GlobalAveragePooling2D,Flatten, Dropout

# Load pre-trained InceptionResNetV2 model, exclude top layers
base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(128 , 128, 3))

# Custom model combining InceptionResNetV2 and BERT embeddings
image_input = base_model.input
x = base_model(image_input)
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)

text_input = Input((768,))
text_input = Flatten()(text_input)

combined = Concatenate()([x, text_input])

x_last = Dense(1024, activation = 'relu')(combined)
x_last = Dense(512, activation = 'relu')(x_last)
x_last = Dropout(0.5)(x_last)
x_last = Dense(256, activation = 'relu')(x_last)
x_last = Dense(128, activation = 'relu')(x_last)
x_last = Dropout(0.5)(x_last)
predictions = Dense(2, activation='softmax')(x_last)


# Define the final model
model = Model(inputs=[image_input, text_input], outputs=predictions)

# Compile the model
model.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])



In [ ]:
model.summary()

In [ ]:
# Train the model
model.fit([train_images, train_text_embeddings], train_labels, epochs=20, batch_size=32,validation_data=([val_images, val_text_embeddings], val_labels))

In [ ]:
predictions = model.predict([val_images, val_text_embeddings])

In [ ]:
predictions = np.argmax(predictions, axis=1)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(val_labels, predictions)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(val_labels, predictions, target_names = ['non_hateful','hateful']))
cm = confusion_matrix(val_labels, predictions)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['non_hateful','hateful'], yticklabels= ['non_hateful','hateful'])
plt.title('Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()